## CS:GO Skin Win probabilities

YouTuber [3kliksphilip](https://www.youtube.com/channel/UCmu9PVIZBk-ZCi-Sk2F2utA) recently posted a video titled ["Case Unboxing Millionaire #2"](https://www.youtube.com/watch?v=a-FdnfZhguw), wherein he showed a single run simulation of him 'unboxing' cases for the first-person-shooter computer game Counter Strike: Global Offensive. These cases are digital items which, when opened, unlock a decorative weapon 'skin'. These skins can be used by players in the game to show off to their friends. Such skins have deveoped quite a market for themselves and can be bought and sold for hundreds or even thousands of dollars through various online marketplaces.

The following analysis calculates the expected profit or loss for a given case, "eSports 2013 Case", per opening. It costs \\$20 to open, and can contain skins worth between a few dollars to around \\$300. 3kliksphilip challenged his viewers to estimate his profit or loss from opening this case up until he unlocked a knife skin, the rarest skin to obatin. I have calculated this expected value to be a loss of \\$5720.96.

In [1]:
import json
import random
import numpy as np

In [2]:
# 'prices_v4.json' downloaded (from https://csgotrader.app/prices/)
with open('prices_v4.json') as f:
  data = json.load(f)

In [3]:
data

{'★ M9 Bayonet | Damascus Steel (Factory New)': {'steam': {'last_90d': 258.27,
   'last_30d': 261.62,
   'last_7d': 263.46,
   'last_24h': 306},
  'bitskins': {'price': '174.5', 'instant_sale_price': '69.8'},
  'lootfarm': 312.08,
  'csgotm': '415.631',
  'csmoney': {'price': 296.85},
  'skincay': {'suggested_price': 178,
   'steam_price': 228.21,
   'instant_price': None,
   'starting_at': None},
  'csgotrader': {'price': 265.61}},
 'StatTrak™ MAC-10 | Stalker (Factory New)': {'steam': {'last_90d': 235.17,
   'last_30d': 247.73,
   'last_7d': 202.54,
   'last_24h': 0},
  'bitskins': {'price': '257.56', 'instant_sale_price': '103.02'},
  'lootfarm': 'null',
  'csgotm': 'null',
  'csmoney': {'price': 280.21},
  'skincay': {'suggested_price': 177.26,
   'steam_price': 227.26,
   'instant_price': None,
   'starting_at': None},
  'csgotrader': {'price': 257.47}},
 'AWP | Fever Dream (Minimal Wear)': {'steam': {'last_90d': 6.93,
   'last_30d': 6.66,
   'last_7d': 6.69,
   'last_24h': 6.7},


In [4]:
# case details (from https://csgostash.com/case/2/eSports-2013-Case)
case_name = "eSports-2013-Case"
case_items = {'Blue': ['FAMAS | Doomkitty', 'MAG-7 | Memento', 'M4A4 | Faded Zebra'],
             'Purple': ['Sawed-Off | Orange DDPAT', 'P250 | Splash', 'Galil AR | Orange DDPAT'],
             'Pink': ['AWP | BOOM', 'AK-47 | Red Laminate'],
             'Red': ['P90 | Death by Kitty'],
             'Yellow': ['Knife']} # can't be bothered putting in all 65 knives, 
                                  # so let's just assume $300 avg value for all knives for our purposes

In [5]:
# implied by float values (from https://totalcsgo.com/skin-conditions)
skin_condition_probs = {'Factory New': 0.07,
                        'Minimal Wear': 0.15,
                        'Field Tested': 0.38,
                        'Well Worn': 0.45,
                        'Battle Scared': 1}
skin_condition_cdf = [0.07, 0.15, 0.38, 0.45, 1]
skin_condition_pdf = [0.07, 0.08, 0.23, 0.07, 0.55]

In [6]:
# from https://www.skinwallet.com/csgo/best-csgo-cases/#Opening_cases_or_buying_skins
case_drop_probs = {'Blue': 1, #0.7992327
                   'Purple': 0.2007672, #0.1598465
                   'Pink': 0.0409207, #0.0319693
                   'Red': 0.0089514, #0.0063939
                   'Yellow': 0.0025575}
case_drop_cdf = [1, 0.2007672, 0.0409207, 0.0089514, 0.0025575]
case_drop_pdf = [0.7992327, 0.1598465, 0.0319693, 0.0063939, 0.0025575]

stattrack_prob = 0.10 # with this probability we'll double the value

In [7]:
# not all items in all conditions are actually available on the marketplace
item = 'P90 | Death by Kitty'
for condition in skin_condition_probs:
    item_condition = item+' ('+condition+')'
    print(item_condition)
    try:
        print(data[item_condition]['steam']['last_90d'])
    except:
        pass

P90 | Death by Kitty (Factory New)
P90 | Death by Kitty (Minimal Wear)
50.55
P90 | Death by Kitty (Field Tested)
P90 | Death by Kitty (Well Worn)
P90 | Death by Kitty (Battle Scared)


In [8]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.
    (from stackoverflow user snake_charmer
    https://stackoverflow.com/questions/6518811/interpolate-nan-values-in-a-numpy-array)

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [9]:
def get_prices(item):
    if item == 'Knife':
        return np.array([300])
    else:
        prices = np.empty(5)
        prices[:] = np.nan
        idx = 0
        for condition in skin_condition_probs:
            item_condition = item+' ('+condition+')'
            try:
                prices[idx] = data[item_condition]['steam']['last_90d']
            except:
                pass
            idx += 1
        if np.count_nonzero(~np.isnan(prices)) == 0: # don't extrapolate if we know nothing
            print("No items of any condition found for {} on the marketplace!".format(item))
        else: # fill extrapolate values
            nans, x = nan_helper(prices)
            prices[nans] = np.interp(x(nans), x(~nans), prices[~nans])
    return prices

In [10]:
case_items_worth = {}
for colour in case_items:
    for item in case_items[colour]:
        case_items_worth[colour] = get_prices(item)

In [11]:
case_items_worth

{'Blue': array([9.13, 1.93, 1.93, 1.93, 1.93]),
 'Purple': array([24.14,  6.77,  6.77,  6.77,  6.77]),
 'Pink': array([143.61,  24.72,  24.72,  24.72,  24.72]),
 'Red': array([50.55, 50.55, 50.55, 50.55, 50.55]),
 'Yellow': array([300])}

In [12]:
expected_item_value = []
for colour in case_items_worth:
    expected_item_value.append(np.sum(case_items_worth[colour] * skin_condition_pdf))
expected_item_value

[2.434, 7.985899999999999, 33.0423, 50.55, 300.0]

In [13]:
expected_case_value = np.sum(np.asarray(expected_item_value) * case_drop_pdf)
expected_case_value

5.368651402539999

In [14]:
case_price = 20
profit_loss = expected_case_value - case_price
expected_number_of_boxes_untill_knife = 1/0.0025575

In [15]:
profit_loss*expected_number_of_boxes_untill_knife

-5720.957418361681